## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fontanka,UA,46.5626,30.8607,45.72,80,93,16.55,overcast clouds
1,1,Ginda,IN,30.6945,78.4932,52.95,36,1,7.94,clear sky
2,2,Bambous Virieux,MU,-20.3428,57.7575,77.22,100,75,11.50,shower rain
3,3,Manjeshwar,IN,12.7194,74.8861,76.15,83,52,4.79,broken clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,46.51,76,40,13.80,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bambous Virieux,MU,-20.3428,57.7575,77.22,100,75,11.50,shower rain
3,3,Manjeshwar,IN,12.7194,74.8861,76.15,83,52,4.79,broken clouds
6,6,Bredasdorp,ZA,-34.5322,20.0403,55.98,81,0,6.76,clear sky
8,8,Yeppoon,AU,-23.1333,150.7333,78.58,61,2,11.48,clear sky
11,11,Hobart,AU,-42.8794,147.3294,69.66,70,0,10.36,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                421
City                   421
Country                418
Lat                    421
Lng                    421
Max Temp               421
Humidity               421
Cloudiness             421
Wind Speed             421
Current Description    421
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                418
City                   418
Country                418
Lat                    418
Lng                    418
Max Temp               418
Humidity               418
Cloudiness             418
Wind Speed             418
Current Description    418
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambous Virieux,MU,77.22,shower rain,-20.3428,57.7575,
3,Manjeshwar,IN,76.15,broken clouds,12.7194,74.8861,
6,Bredasdorp,ZA,55.98,clear sky,-34.5322,20.0403,
8,Yeppoon,AU,78.58,clear sky,-23.1333,150.7333,
11,Hobart,AU,69.66,moderate rain,-42.8794,147.3294,
12,East London,ZA,62.06,overcast clouds,-33.0153,27.9116,
13,Mahebourg,MU,77.25,shower rain,-20.4081,57.7000,
14,Lakes Entrance,AU,69.21,scattered clouds,-37.8811,147.9810,
15,Diffa,NE,87.37,few clouds,13.3154,12.6113,
16,Ypsonas,CY,66.27,clear sky,34.6884,32.9574,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]    
    except (IndexError):
        print("Hotel not found, skipping.")
        


Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.
Hotel not found, skipping.


In [76]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambous Virieux,MU,77.22,shower rain,-20.3428,57.7575,Casa Tia Villa
3,Manjeshwar,IN,76.15,broken clouds,12.7194,74.8861,Mangalore Beach House - EWA Ikigai
6,Bredasdorp,ZA,55.98,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
8,Yeppoon,AU,78.58,clear sky,-23.1333,150.7333,Villa Mar Colina
11,Hobart,AU,69.66,moderate rain,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania


In [75]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPY_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [90]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [91]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))